In [38]:
import sys
import numpy as np
import tensorflow as tf
from tensorflow.python.framework import ops
import matplotlib.pyplot as plt
sys.path.append("../Functions/")
# sys.path.remove("../Functions/")
from getImageArrays import getExamples
import getImageArrays
from importlib import reload

In [25]:
#reloads source files to get most recent edits
import importlib
importlib.reload(getImageArrays)
from getImageArrays import getExamples

In [26]:
image_path = "../Data/Logistic_Regression_Data/"
num_images = 100
examples_per_batch = 16

In [27]:
train_x, train_y, test_x, test_y = getExamples(side_length=100, image_path=image_path , test_ratio=.2, max_num_images=num_images)
# train_x, train_y, test_x, test_y  = train_x.T, train_y.T, test_x.T, test_y.T
assert((len(train_x) + len(test_x)) == num_images), "Not sufficeint number of images" 

getExamples!
examplesCow (50, 100, 100, 3)
../Data/Logistic_Regression_Data/notcows/ herererer
Invalid shape notcow197.jpg
examplesNotCow (50, 100, 100, 3)
finished getImageArrays!
lables_cow (50, 2)
lables_notCow (50, 2)
Examples Before (100, 100, 100, 3)
labels Before (100, 2)
Examples After (100, 100, 100, 3)
labels After (100, 2)


In [28]:
def createBatches(x, examples_per_batch):
    final = []
    for i in range(0, len(x), examples_per_batch):
        start = i
        end = start + examples_per_batch
        #creates exclusively
        if end >= len(x):
            return np.array(final)
        final += [x[start:end]]
    return np.array(final)

In [29]:
train_x, train_y = createBatches(train_x, examples_per_batch), createBatches(train_y, examples_per_batch)
test_x, test_y  = createBatches(test_x, examples_per_batch), createBatches(test_y, examples_per_batch)

In [30]:
print("train_x", train_x.shape)
print("train_y", train_y.shape)
print("test_x", test_x.shape)
print("test_y", test_y.shape)

train_x (4, 16, 100, 100, 3)
train_y (4, 16, 2)
test_x (1, 16, 100, 100, 3)
test_y (1, 16, 2)


In [8]:
np.random.seed(1)
train_x = np.random.randn(1080, 64, 64, 3)
train_y = np.random.randn(1080, 6)
test_x = np.random.randn(120, 64, 64, 3)
test_y = np.random.randn(120, 6)

In [13]:
def create_placeholders(width, height, channels, num_output_classes):
    placeholder_x = tf.placeholder(tf.float32, shape=(None, width, height, channels))
    placeholder_y = tf.placeholder(tf.float32, shape=(None, num_output_classes))
    return placeholder_x, placeholder_y

In [14]:
placeholder_x, placeholder_y = create_placeholders(64, 64, 3, 2)
print(placeholder_x)
print(placeholder_y)

Tensor("Placeholder:0", shape=(?, 64, 64, 3), dtype=float32)
Tensor("Placeholder_1:0", shape=(?, 2), dtype=float32)


In [15]:
def initialize_params():
    tf.set_random_seed(1)
    #shape = (window_height, window_width, prev_channels, new_channels)
    W1 = tf.get_variable("W1", shape=[4, 4, 3, 8], initializer=tf.contrib.layers.xavier_initializer(seed=0))
    W2 = tf.get_variable("W2", shape=[2, 2, 8, 16], initializer=tf.contrib.layers.xavier_initializer(seed=0))
    #tf takes care of biases
    return {"W1": W1,"W2":W2}

In [33]:
tf.reset_default_graph()
with tf.Session() as sess:
    params = initialize_params()
    initialize = tf.global_variables_initializer()
    sess.run(initialize)
    print(params["W1"].eval()[1, 1, 1])
    print(params["W2"].eval()[1, 1, 1])

[ 0.00131723  0.14176141 -0.04434952  0.09197326  0.14984085 -0.03514394
 -0.06847463  0.05245192]
[-0.08566415  0.17750949  0.11974221  0.16773748 -0.0830943  -0.08058
 -0.00577033 -0.14643836  0.24162132 -0.05857408 -0.19055021  0.1345228
 -0.22779644 -0.1601823  -0.16117483 -0.10286498]


[ 0.00131723  0.14176141 -0.04434952  0.09197326  0.14984085 -0.03514394
 -0.06847463  0.05245192]
[-0.08566415  0.17750949  0.11974221  0.16773748 -0.0830943  -0.08058
 -0.00577033 -0.14643836  0.24162132 -0.05857408 -0.19055021  0.1345228
 -0.22779644 -0.1601823  -0.16117483 -0.10286498]

In [17]:
def foward_propagate(x, params):
    #tf useses params deduce meta data about architecture
    W1 = params["W1"]
    W2 = params["W2"]
    
    # Layer 1: Convolution -> Relu -> Max Pool
    z1 = tf.nn.conv2d(x, W1, strides=[1,1,1,1], padding="SAME")
    a1 = tf.nn.relu(z1)
    p1 = tf.nn.max_pool(a1, ksize=[1, 8, 8, 1], strides=[1, 8, 8, 1], padding="SAME")
    
    # Layer 2: Convolution -> Relu -> Max Pool
    z2 = tf.nn.conv2d(p1, W2, strides=[1,1,1,1], padding="SAME")
    a2 = tf.nn.relu(z2)
    p2 = tf.nn.max_pool(a2, ksize=[1, 4, 4, 1], strides=[1, 4, 4, 1], padding="SAME")
    
    # 
    flatten_cnn_output = tf.contrib.layers.flatten(p2)
    preds = tf.contrib.layers.fully_connected(flatten_cnn_output, 2, activation_fn=None)
    
    return preds    

In [17]:
# np.random.seed(17)
tf.reset_default_graph()
with tf.Session() as sess:
    np.random.seed(1)
    placeholder_x, placeholder_y = create_placeholders(64, 64, 3, 6)
    params = initialize_params()
    preds = foward_propagate(placeholder_x, params)
    initialize = tf.global_variables_initializer()
    sess.run(initialize)
    test = sess.run(preds, {placeholder_x: np.random.randn(2, 64, 64, 3), placeholder_y: np.random.randn(2, 6)})
print(test)

[[ 1.44169843 -0.24909666  5.45049906 -0.26189619 -0.20669907  1.36546707]
 [ 1.40708458 -0.02573211  5.08928013 -0.48669922 -0.40940708  1.26248586]]


[[ 1.44169843 -0.24909666  5.45049906 -0.26189619 -0.20669907  1.36546707]
 [ 1.40708458 -0.02573211  5.08928013 -0.48669922 -0.40940708  1.26248586]]

In [18]:
def cost_op(preds, y):
    costs = tf.nn.softmax_cross_entropy_with_logits(logits=preds, labels=y)
    #gets average
    avg_cost = tf.reduce_mean(costs)
#     avg_cost = tf.abs(avg_cost)
#     costs = tf.abs(costs)
#     avg_cost = tf.reduce_mean(costs)
    print("COSTS", costs)
    print("AVG Cost", avg_cost)
    return avg_cost

In [92]:
# np.random.seed(17)
tf.reset_default_graph()
with tf.Session() as sess:
    np.random.seed(1)
    placeholder_x, placeholder_y = create_placeholders(64, 64, 3, 6)
    params = initialize_params()
    preds = foward_propagate(placeholder_x, params)
    print(preds.shape)
    avg_cost = cost_op(preds, placeholder_y)
    
    initialize = tf.global_variables_initializer()
    sess.run(initialize)
    
    test = sess.run(avg_cost, {placeholder_x: np.random.randn(4, 64, 64, 3), placeholder_y: np.random.randn(4, 6)})
print(test)

(?, 1)


InvalidArgumentError: logits and labels must be same size: logits_size=[4,1] labels_size=[4,6]
	 [[Node: softmax_cross_entropy_with_logits_sg = SoftmaxCrossEntropyWithLogits[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"](softmax_cross_entropy_with_logits_sg/Reshape, softmax_cross_entropy_with_logits_sg/Reshape_1)]]

Caused by op 'softmax_cross_entropy_with_logits_sg', defined at:
  File "/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/anaconda3/lib/

In [46]:
def model(trainX, trainY, testX, testY, epoch, lr, output_num_classes):
    ops.reset_default_graph()
#     tf.set_random_seed(17)
    num_batches, num_train_examples , width, height, num_channels = trainX.shape
    _, num_test_examples , _, _, _ = testX.shape
#     output_num_classes = 1
    costs = []
    
    # Conv -> ....
    placeholder_x, placeholder_y = create_placeholders(width, height, num_channels, output_num_classes)
    params = initialize_params()
    preds = foward_propagate(placeholder_x, params)
    spred = (placeholder_y, preds)
    avg_cost = cost_op(preds, placeholder_y)
    optimizer = tf.train.AdamOptimizer(lr).minimize(avg_cost)
    
    initialize = tf.global_variables_initializer()
    with tf.Session() as sess:
        sess.run(initialize)
        
        for i in range(epoch):
                #remember to shuffle batches
            for b in range(num_batches):
    #         mini_batch_cost = 0
    #         num_mini_batches = int(num_train_examples/batch_size)

    #         mini_batch_cost = sess.run(avg_cost, {placeholder_x: np.random.randn(4, 64, 64, 3), placeholder_y: np.random.randn(4, 6)})
    #         costs += mini_batch_cost
    

                cur_cost = sess.run([optimizer, preds, avg_cost, spred], {placeholder_x: trainX[b], placeholder_y: trainY[b]})
#                 print("preds", preds)
                costs += [cur_cost[2]]
            if i % 20 == 0:
                print("Avg recent costs", sum(costs[-20:]) / 20)
#                 print("Spred", cur_cost[3])
        predict = tf.argmax(preds, 1)
        true_label = tf.argmax(placeholder_y, 1)
        correct = tf.equal(predict, true_label)
    
        accuracy = tf.reduce_mean(tf.cast(correct, "float"))
        train_acc = accuracy.eval({placeholder_x: trainX[0], placeholder_y: trainY[0]})
        test_acc = accuracy.eval({placeholder_x: testX[0], placeholder_y: testY[0]})
        print("accuracy", accuracy)
        print("train_acc", train_acc)
        print("test_acc", test_acc)
    
    x = [i for i in range(len(costs))]
    y = costs
    plt.plot(x, y)
    plt.ylabel("cost")
    plt.xlabel("iters")
#     plt.titl
    plt.show()
    
    return sess
    
#     return train_acc, test_acc, params

In [41]:
print("train_x", train_x.shape)
print("train_y", train_y.shape)
print("test_x", test_x.shape)
print("test_y", test_y.shape)

train_x (4, 16, 100, 100, 3)
train_y (4, 16, 2)
test_x (1, 16, 100, 100, 3)
test_y (1, 16, 2)


In [ ]:
sess = model(train_x, train_y, test_x, test_y, 100, .005, 2)

COSTS Tensor("Reshape_2:0", shape=(?,), dtype=float32)
AVG Cost Tensor("Mean:0", shape=(), dtype=float32)
Avg recent costs 0.137735959888
Avg recent costs 0.103109914809
Avg recent costs 0.00856989331078
Avg recent costs 0.00266801022808
Avg recent costs 0.00131443661812
